# Project Strongest - 初期データ分析 (EDA)

このNotebookでは、`JraVanDataLoader` を介してロードされたデータセットの初期概要を確認します。

In [ ]:
import sys
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# notebooksディレクトリから実行している場合、srcをパスに追加
if 'src' not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), '../src'))

from preprocessing.loader import JraVanDataLoader

%matplotlib inline
# 日本語フォント設定（環境に合わせて適宜変更してください。Windowsの場合はMS Gothicなど）
sns.set(font='MSGothic') 

## 1. データのロード
`JraVanDataLoader` を使用してデータをロードします。PostgreSQLデータベースに接続し、レースおよび結果情報を取得します。

In [ ]:
try:
    loader = JraVanDataLoader()
    # EDA用に10万件をロードします。必要に応じてlimitを調整してください。
    df = loader.load(limit=100000)
    print(f"{len(df)} 行のデータを正常にロードしました")
except Exception as e:
    print(f"データのロード中にエラーが発生しました: {e}")
    print("Dockerコンテナ内で実行しているか、DB接続設定が正しいことを確認してください。")

In [ ]:
df.head()

In [ ]:
df.info()

## 2. 基本統計量

In [ ]:
df[['odds', 'popularity', 'weight', 'age', 'distance']].describe()

## 3. 可視化

### 3.1 着順の分布
着順データの偏りや欠損がないか確認します。

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='rank')
plt.title('着順の分布')
plt.xlabel('着順')
plt.ylabel('件数')
plt.show()

### 3.2 単勝オッズの分布
単勝オッズの分布を可視化します。通常はべき乗則に従います。

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(df[df['odds'] < 100]['odds'], bins=100, kde=True)
plt.title('単勝オッズの分布 (100倍未満)')
plt.xlabel('オッズ')
plt.show()

### 3.3 相関行列
数値特徴量と着順（Rank）の相関を確認します。

In [ ]:
plt.figure(figsize=(10, 8))
numeric_cols = ['rank', 'odds', 'popularity', 'weight', 'age', 'distance', 'last_3f']
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('相関行列')
plt.show()